In [10]:
from platform import python_version
import pkg_resources

In [11]:
import torch
import torch.nn as nn
import check_pypi_name as check
from torchvision import models
from torchsummary import summary
import torch.optim as optim
import time
import read_data
from collections import OrderedDict 
import matplotlib.pyplot as plt
import numpy as np

In [28]:
from platform import python_version
print(python_version())

3.6.12


Vamos a crear un pequeño script, que nos permite leer la versión del programa del que estemos interesados conocer

In [12]:
def version_library(programa):
    return pkg_resources.get_distribution(programa).version

In [22]:
programa='torch'
version_library(programa)

'1.6.0'

In [20]:
programa='numpy'
version_library(programa)

'1.19.5'

In [25]:
programa='check_pypi_name'
version_library(programa)

'0.4.0'

In [26]:
programa='matplotlib'
version_library(programa)

'3.3.2'

In [27]:
programa='numpy'
version_library(programa)

'1.19.5'

# Device

Podemos colocar la información de los tensores en el GPU, si es que contamos con uno. Para ello, vamos a crear el siguiente programa, para identificar si tenemos instalado en nuestra computadora alguna GPU

In [41]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

veamos qué obtenemos

In [42]:
device=get_device()

In [43]:
device

device(type='cpu')

Veamos cómo podemos introducir nuestro device en el cálculo de un tensor

In [35]:
prueba=torch.tensor([[3.0,4.0,5.0],[1.0, 3.0, 7.0]], device=device)

Esta manera de definir el device de nuestros tensores puede parecer no tan amigable, en el sentido de que si por alguna razón necesitamos cambiar nuestro almacenamiento, tendríamos que modificar manualmente la propiedad de device a todos los tensores de nuestro trabajo. Entonces, lo que podemos hacer, es definir la siguiente función, que nos manda tensores, elementos o tuplas de ellos, al device con el que estemos trabajando en ese momento

In [36]:
def to_device(data, device):
    if isinstance(data, (list,tuple)): #nos regresa True o False, si el data es del tipo especificado en la segunda entrada
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

Veamos un ejemplo sencillo

In [45]:
ejemplo=torch.randn(3,2)
ejemplo=to_device(ejemplo, device)
ejemplo.device

device(type='cpu')

Y con esta función, podemos definir otra más compleja, que nos permita mover todo un batch, generado por la función DataLoader de Pytorch

In [40]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

así, podemos mandar todos los elementos de un batch al device que estemos trabajando

# Ventajas de trabajar con device definido en todos nuestros tensores

Conforme revisemos la literatura y los códigos que nos podremos encontrar a lo largo del curso, habrá autores que prefieran trabajar con CPU o GPU. Y teniendo estos pequeños fragmentos de código, nos permitirán movernos sim problemas entre estos estilos, y correremos los programas sin mayor dificultad